In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
from scipy import stats


In [2]:
def sample_distribution(dist_type, dist_category, n_samples, **params):
    """Sample from the specified distribution"""
    if dist_category == "Continuous":
        if dist_type == "Normal":
            mean = params.get('mean', 0)
            std = params.get('std', 1)
            return np.random.normal(mean, std, n_samples)
        elif dist_type == "Exponential":
            scale = params.get('scale', 1)
            return np.random.exponential(scale, n_samples)
        elif dist_type == "Beta":
            alpha = params.get('alpha', 2)
            beta = params.get('beta', 2)
            return np.random.beta(alpha, beta, n_samples)
        elif dist_type == "Gamma":
            shape = params.get('shape', 2)
            scale = params.get('scale', 1)
            return np.random.gamma(shape, scale, n_samples)
        elif dist_type == "Uniform":
            low = params.get('low', 0)
            high = params.get('high', 1)
            return np.random.uniform(low, high, n_samples)
        else:
            return np.random.normal(0, 1, n_samples)
    else:  # Discrete
        if dist_type == "Poisson":
            lam = params.get('lam', 5)
            return np.random.poisson(lam, n_samples)
        elif dist_type == "Binomial":
            n = params.get('n', 10)
            p = params.get('p', 0.5)
            return np.random.binomial(n, p, n_samples)
        elif dist_type == "Geometric":
            p = params.get('p', 0.5)
            return np.random.geometric(p, n_samples)
        elif dist_type == "Negative Binomial":
            n = params.get('n', 5)
            p = params.get('p', 0.5)
            return np.random.negative_binomial(n, p, n_samples)
        else:
            return np.random.poisson(5, n_samples)


In [3]:
def compute_pdf_pmf(x_values, dist_type, dist_category, **params):
    """Compute PDF (continuous) or PMF (discrete) for the distribution"""
    if dist_category == "Continuous":
        if dist_type == "Normal":
            mean = params.get('mean', 0)
            std = params.get('std', 1)
            return stats.norm.pdf(x_values, mean, std)
        elif dist_type == "Exponential":
            scale = params.get('scale', 1)
            return stats.expon.pdf(x_values, scale=scale)
        elif dist_type == "Beta":
            alpha = params.get('alpha', 2)
            beta = params.get('beta', 2)
            return stats.beta.pdf(x_values, alpha, beta)
        elif dist_type == "Gamma":
            shape = params.get('shape', 2)
            scale = params.get('scale', 1)
            return stats.gamma.pdf(x_values, shape, scale=scale)
        elif dist_type == "Uniform":
            low = params.get('low', 0)
            high = params.get('high', 1)
            return stats.uniform.pdf(x_values, loc=low, scale=high-low)
        else:
            return stats.norm.pdf(x_values, 0, 1)
    else:  # Discrete
        if dist_type == "Poisson":
            lam = params.get('lam', 5)
            # For discrete, return PMF at integer values
            return stats.poisson.pmf(np.round(x_values).astype(int), lam)
        elif dist_type == "Binomial":
            n = params.get('n', 10)
            p = params.get('p', 0.5)
            return stats.binom.pmf(np.round(x_values).astype(int), n, p)
        elif dist_type == "Geometric":
            p = params.get('p', 0.5)
            return stats.geom.pmf(np.round(x_values).astype(int), p)
        elif dist_type == "Negative Binomial":
            n = params.get('n', 5)
            p = params.get('p', 0.5)
            return stats.nbinom.pmf(np.round(x_values).astype(int), n, p)
        else:
            return stats.poisson.pmf(np.round(x_values).astype(int), 5)


In [4]:
def compute_true_probability(dist_type, dist_category, prob_type, bound1, bound2, **params):
    """Compute true probability using CDF/PMF"""
    if dist_category == "Continuous":
        if dist_type == "Normal":
            mean = params.get('mean', 0)
            std = params.get('std', 1)
            dist = stats.norm(mean, std)
        elif dist_type == "Exponential":
            scale = params.get('scale', 1)
            dist = stats.expon(scale=scale)
        elif dist_type == "Beta":
            alpha = params.get('alpha', 2)
            beta = params.get('beta', 2)
            dist = stats.beta(alpha, beta)
        elif dist_type == "Gamma":
            shape = params.get('shape', 2)
            scale = params.get('scale', 1)
            dist = stats.gamma(shape, scale=scale)
        elif dist_type == "Uniform":
            low = params.get('low', 0)
            high = params.get('high', 1)
            dist = stats.uniform(loc=low, scale=high-low)
        else:
            dist = stats.norm(0, 1)
        
        if prob_type == "of outcome":
            # For continuous, P(X = x) = 0, so return 0
            return 0.0
        elif prob_type == "under upper bound":
            return dist.cdf(bound2)  # Use upper bound (bound2)
        elif prob_type == "above lower bound":
            return 1 - dist.cdf(bound1)
        elif prob_type == "in interval":
            return dist.cdf(bound2) - dist.cdf(bound1)
    else:  # Discrete
        if dist_type == "Poisson":
            lam = params.get('lam', 5)
            dist = stats.poisson(lam)
        elif dist_type == "Binomial":
            n = params.get('n', 10)
            p = params.get('p', 0.5)
            dist = stats.binom(n, p)
        elif dist_type == "Geometric":
            p = params.get('p', 0.5)
            dist = stats.geom(p)
        elif dist_type == "Negative Binomial":
            n = params.get('n', 5)
            p = params.get('p', 0.5)
            dist = stats.nbinom(n, p)
        else:
            dist = stats.poisson(5)
        
        if prob_type == "of outcome":
            return dist.pmf(int(np.round(bound1)))
        elif prob_type == "under upper bound":
            return dist.cdf(int(np.round(bound2)))  # Use upper bound (bound2)
        elif prob_type == "above lower bound":
            return 1 - dist.cdf(int(np.round(bound1)) - 1)  # -1 because we want > bound1
        elif prob_type == "in interval":
            return dist.cdf(int(np.round(bound2))) - dist.cdf(int(np.round(bound1)) - 1)
    
    return 0.0


In [5]:
def compute_estimated_probability(samples, prob_type, bound1, bound2):
    """Compute estimated probability from samples"""
    if prob_type == "of outcome":
        # Count samples exactly equal to bound1 (for discrete, use integer comparison)
        if samples.dtype in [np.int32, np.int64] or np.all(samples == np.round(samples)):
            count = np.sum(samples == int(np.round(bound1)))
        else:
            count = np.sum(np.abs(samples - bound1) < 1e-6)
    elif prob_type == "under upper bound":
        count = np.sum(samples <= bound2)  # Use upper bound (bound2)
    elif prob_type == "above lower bound":
        count = np.sum(samples > bound1)
    elif prob_type == "in interval":
        count = np.sum((samples > bound1) & (samples <= bound2))
    else:
        return 0.0
    
    return count / len(samples) if len(samples) > 0 else 0.0


In [6]:
class DistributionProbabilityVisualization:
    """Interactive visualization for distribution sampling and probability calculation"""
    
    def __init__(self):
        self.samples = np.array([])
        self.plot_output = widgets.Output()
        self.show_pdf_flag = False  # Track whether PDF should be shown
        
        # Distribution options
        self.continuous_dists = ["Normal", "Exponential", "Beta", "Gamma", "Uniform"]
        self.discrete_dists = ["Poisson", "Binomial", "Geometric", "Negative Binomial"]
        
        self._create_widgets()
        self._setup_callbacks()
        
    def _create_widgets(self):
        """Create all widgets"""
        # Category dropdown
        self.category_dropdown = widgets.Dropdown(
            options=["Continuous", "Discrete"],
            value="Continuous",
            description="Type:",
            style={'description_width': 'initial'}
        )
        
        # Distribution dropdown (will be updated based on category)
        self.dist_dropdown = widgets.Dropdown(
            options=self.continuous_dists,
            value="Normal",
            description="Distribution:",
            style={'description_width': 'initial'}
        )
        
        # Parameter widgets (will be shown/hidden based on distribution)
        self.param_widgets = {}
        
        # Normal parameters
        self.param_widgets['Normal'] = [
            widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Mean:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=1, min=0.1, max=3, step=0.1, description='Std:', style={'description_width': 'initial'})
        ]
        
        # Exponential parameters
        self.param_widgets['Exponential'] = [
            widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Scale:', style={'description_width': 'initial'})
        ]
        
        # Beta parameters
        self.param_widgets['Beta'] = [
            widgets.FloatSlider(value=2, min=0.5, max=10, step=0.1, description='Alpha:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=2, min=0.5, max=10, step=0.1, description='Beta:', style={'description_width': 'initial'})
        ]
        
        # Gamma parameters
        self.param_widgets['Gamma'] = [
            widgets.FloatSlider(value=2, min=0.5, max=10, step=0.1, description='Shape:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Scale:', style={'description_width': 'initial'})
        ]
        
        # Uniform parameters
        self.param_widgets['Uniform'] = [
            widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Low:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='High:', style={'description_width': 'initial'})
        ]
        
        # Poisson parameters
        self.param_widgets['Poisson'] = [
            widgets.FloatSlider(value=5, min=0.5, max=20, step=0.1, description='Lambda:', style={'description_width': 'initial'})
        ]
        
        # Binomial parameters
        self.param_widgets['Binomial'] = [
            widgets.IntSlider(value=10, min=1, max=50, step=1, description='n:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=0.5, min=0.1, max=0.9, step=0.05, description='p:', style={'description_width': 'initial'})
        ]
        
        # Geometric parameters
        self.param_widgets['Geometric'] = [
            widgets.FloatSlider(value=0.5, min=0.1, max=0.9, step=0.05, description='p:', style={'description_width': 'initial'})
        ]
        
        # Negative Binomial parameters
        self.param_widgets['Negative Binomial'] = [
            widgets.IntSlider(value=5, min=1, max=20, step=1, description='n:', style={'description_width': 'initial'}),
            widgets.FloatSlider(value=0.5, min=0.1, max=0.9, step=0.05, description='p:', style={'description_width': 'initial'})
        ]
        
        # Sample size
        self.n_samples_slider = widgets.IntSlider(
            value=1000, min=100, max=10000, step=100,
            description="Samples:",
            style={'description_width': 'initial'}
        )
        
        # Draw samples button
        self.draw_button = widgets.Button(
            description="Draw Samples",
            button_style='success'
        )
        
        # Reset all button
        self.reset_button = widgets.Button(
            description="Reset all",
            button_style='warning'
        )
        
        # Show PDF/PMF button (disabled initially)
        self.show_pdf_button = widgets.Button(
            description="Show PDF/PMF",
            button_style='info',
            disabled=True  # Disabled until samples are drawn
        )
        
        # Probability calculation dropdown
        self.prob_type_dropdown = widgets.Dropdown(
            options=["of outcome", "under upper bound", "above lower bound", "in interval"],
            value="in interval",
            description="Find Probability:",
            style={'description_width': 'initial'}
        )
        
        # Bound sliders (will be shown/hidden based on prob_type)
        self.bound1_slider = widgets.FloatSlider(
            value=0, min=-10, max=10, step=0.1,
            description="Lower bound:",
            style={'description_width': 'initial'}
        )
        
        self.bound2_slider = widgets.FloatSlider(
            value=1, min=-10, max=10, step=0.1,
            description="Upper bound:",
            style={'description_width': 'initial'}
        )
        
        # Probability display
        self.prob_label = widgets.HTML(
            value="<b>Estimated Probability:</b> N/A<br><b>True Probability:</b> N/A"
        )
        
        # Parameter container (will be updated)
        self.param_container = widgets.VBox([])
        
        # Probability controls container (initially hidden)
        self.prob_controls_container = widgets.VBox([
            widgets.HTML("<hr>"),
            self.prob_type_dropdown,
            self.bound1_slider,
            self.bound2_slider,
            self.show_pdf_button,  # Show PDF/PMF button directly under Upper bound
            widgets.HTML("<hr>"),
            self.prob_label
        ])
        # Initially hide the probability controls
        self.prob_controls_container.layout.display = 'none'
        
    def _setup_callbacks(self):
        """Setup widget callbacks"""
        self.category_dropdown.observe(self._on_category_change, names='value')
        self.dist_dropdown.observe(self._on_dist_change, names='value')
        self.prob_type_dropdown.observe(self._on_prob_type_change, names='value')
        self.draw_button.on_click(self._on_draw_clicked)
        self.reset_button.on_click(self._on_reset_clicked)
        self.show_pdf_button.on_click(self._on_show_pdf_clicked)
        
        # Update plot when sliders change (but only if samples exist)
        for widgets_list in self.param_widgets.values():
            for w in widgets_list:
                w.observe(self._on_param_change, names='value')
        
        self.n_samples_slider.observe(self._on_param_change, names='value')
        self.bound1_slider.observe(self._on_bound_change, names='value')
        self.bound2_slider.observe(self._on_bound_change, names='value')
        
    def _on_bound_change(self, change):
        """Handle bound slider changes - only update plot if samples exist"""
        if len(self.samples) > 0:
            self._update_plot(show_pdf=self.show_pdf_flag)  # Use current PDF flag state
        
    def _on_param_change(self, change):
        """Handle parameter changes - only update plot if samples exist"""
        if len(self.samples) > 0:
            self._update_plot(show_pdf=self.show_pdf_flag)  # Use current PDF flag state
        
    def _on_category_change(self, change):
        """Handle category change"""
        if change['new'] == "Continuous":
            self.dist_dropdown.options = self.continuous_dists
            self.dist_dropdown.value = "Normal"
        else:
            self.dist_dropdown.options = self.discrete_dists
            self.dist_dropdown.value = "Poisson"
        self._update_param_widgets()
        # Clear samples and show blank plot
        self.samples = np.array([])
        self.show_pdf_flag = False
        self.show_pdf_button.disabled = True
        self.show_pdf_button.description = "Show PDF/PMF"
        # Hide probability controls
        self.prob_controls_container.layout.display = 'none'
        self._show_blank_plot()
        
    def _on_dist_change(self, change):
        """Handle distribution change"""
        self._update_param_widgets()
        # Clear samples and show blank plot
        self.samples = np.array([])
        self.show_pdf_flag = False
        self.show_pdf_button.disabled = True
        self.show_pdf_button.description = "Show PDF/PMF"
        # Hide probability controls
        self.prob_controls_container.layout.display = 'none'
        self._show_blank_plot()
        
    def _on_reset_clicked(self, button):
        """Handle Reset all button click - reset everything to initial state"""
        # Clear samples
        self.samples = np.array([])
        
        # Reset PDF flag and button
        self.show_pdf_flag = False
        self.show_pdf_button.disabled = True
        self.show_pdf_button.description = "Show PDF/PMF"
        
        # Reset probability label
        self.prob_label.value = "<b>Estimated Probability:</b> N/A<br><b>True Probability:</b> N/A"
        
        # Hide probability controls
        self.prob_controls_container.layout.display = 'none'
        
        # Show blank plot
        self._show_blank_plot()
        
    def _on_show_pdf_clicked(self, button):
        """Handle Show PDF/PMF button click"""
        if len(self.samples) > 0:
            self.show_pdf_flag = not self.show_pdf_flag
            if self.show_pdf_flag:
                self.show_pdf_button.description = "Hide PDF/PMF"
            else:
                self.show_pdf_button.description = "Show PDF/PMF"
            self._update_plot(show_pdf=self.show_pdf_flag)
        
    def _update_param_widgets(self):
        """Update parameter widgets based on current distribution"""
        dist = self.dist_dropdown.value
        if dist in self.param_widgets:
            self.param_container.children = tuple(self.param_widgets[dist])
        # Don't update plot here to avoid double updates
        
    def _on_prob_type_change(self, change):
        """Handle probability type change"""
        if len(self.samples) > 0:
            self._update_plot(show_pdf=self.show_pdf_flag)  # Use current PDF flag state
        
    def _on_draw_clicked(self, button):
        """Handle draw samples button"""
        self._draw_samples()
        self._update_bound_sliders()
        # Enable the Show PDF/PMF button
        self.show_pdf_button.disabled = False
        self.show_pdf_flag = False  # Reset to not showing PDF initially
        self.show_pdf_button.description = "Show PDF/PMF"
        # Show probability controls
        self.prob_controls_container.layout.display = 'flex'
        self._update_plot(show_pdf=False)  # Only show histogram initially
        
    def _update_bound_sliders(self):
        """Update bound slider ranges based on current samples"""
        if len(self.samples) > 0:
            x_min = float(np.min(self.samples)) - 2
            x_max = float(np.max(self.samples)) + 2
            
            dist_category = self.category_dropdown.value
            if dist_category == "Discrete":
                x_min = max(0, int(x_min))
                x_max = int(x_max) + 1
                step = 1
            else:
                step = 0.1
            
            # Update sliders
            self.bound1_slider.min = x_min
            self.bound1_slider.max = x_max
            self.bound1_slider.step = step
            if self.bound1_slider.value < x_min:
                self.bound1_slider.value = x_min
            elif self.bound1_slider.value > x_max:
                self.bound1_slider.value = x_max
            
            self.bound2_slider.min = x_min
            self.bound2_slider.max = x_max
            self.bound2_slider.step = step
            if self.bound2_slider.value < x_min:
                self.bound2_slider.value = x_min
            elif self.bound2_slider.value > x_max:
                self.bound2_slider.value = x_max
        
    def _draw_samples(self):
        """Draw new samples"""
        dist_type = self.dist_dropdown.value
        dist_category = self.category_dropdown.value
        n_samples = self.n_samples_slider.value
        
        # Get parameters
        params = {}
        if dist_type in self.param_widgets:
            widgets_list = self.param_widgets[dist_type]
            if dist_type == "Normal":
                params['mean'] = widgets_list[0].value
                params['std'] = widgets_list[1].value
            elif dist_type == "Exponential":
                params['scale'] = widgets_list[0].value
            elif dist_type == "Beta":
                params['alpha'] = widgets_list[0].value
                params['beta'] = widgets_list[1].value
            elif dist_type == "Gamma":
                params['shape'] = widgets_list[0].value
                params['scale'] = widgets_list[1].value
            elif dist_type == "Uniform":
                params['low'] = widgets_list[0].value
                params['high'] = widgets_list[1].value
            elif dist_type == "Poisson":
                params['lam'] = widgets_list[0].value
            elif dist_type == "Binomial":
                params['n'] = widgets_list[0].value
                params['p'] = widgets_list[1].value
            elif dist_type == "Geometric":
                params['p'] = widgets_list[0].value
            elif dist_type == "Negative Binomial":
                params['n'] = widgets_list[0].value
                params['p'] = widgets_list[1].value
        
        self.samples = sample_distribution(dist_type, dist_category, n_samples, **params)
        
    def _get_params_dict(self):
        """Get current parameters as dictionary"""
        dist_type = self.dist_dropdown.value
        params = {}
        if dist_type in self.param_widgets:
            widgets_list = self.param_widgets[dist_type]
            if dist_type == "Normal":
                params['mean'] = widgets_list[0].value
                params['std'] = widgets_list[1].value
            elif dist_type == "Exponential":
                params['scale'] = widgets_list[0].value
            elif dist_type == "Beta":
                params['alpha'] = widgets_list[0].value
                params['beta'] = widgets_list[1].value
            elif dist_type == "Gamma":
                params['shape'] = widgets_list[0].value
                params['scale'] = widgets_list[1].value
            elif dist_type == "Uniform":
                params['low'] = widgets_list[0].value
                params['high'] = widgets_list[1].value
            elif dist_type == "Poisson":
                params['lam'] = widgets_list[0].value
            elif dist_type == "Binomial":
                params['n'] = widgets_list[0].value
                params['p'] = widgets_list[1].value
            elif dist_type == "Geometric":
                params['p'] = widgets_list[0].value
            elif dist_type == "Negative Binomial":
                params['n'] = widgets_list[0].value
                params['p'] = widgets_list[1].value
        return params
        
    def _show_blank_plot(self):
        """Show blank plot when distribution changes"""
        with self.plot_output:
            clear_output(wait=True)
            # Create empty figure with two subplots (histogram on top)
            fig = make_subplots(
                rows=2, cols=1,
                row_heights=[0.3, 0.7],  # Histogram on top (30%), PDF on bottom (70%)
                vertical_spacing=0.15,
                shared_xaxes=True,
                subplot_titles=('Histogram', 'PDF/PMF and Samples')
            )
            # Set default axis ranges for blank plot
            fig.update_xaxes(title_text="x", range=[-5, 5], row=1, col=1)
            fig.update_xaxes(title_text="x", range=[-5, 5], row=2, col=1)
            fig.update_yaxes(title_text="Density", range=[0, 1], row=1, col=1)
            fig.update_yaxes(title_text="Density", range=[0, 1], row=2, col=1)
            fig.update_layout(height=700, showlegend=True)
            fig.show()
            
            # Reset probability label
            self.prob_label.value = "<b>Estimated Probability:</b> N/A<br><b>True Probability:</b> N/A"
    
    def _update_plot(self, change=None, show_pdf=True):
        """Update the plot"""
        if len(self.samples) == 0:
            self._show_blank_plot()
            return
            
        with self.plot_output:
            clear_output(wait=True)
            
            dist_type = self.dist_dropdown.value
            dist_category = self.category_dropdown.value
            prob_type = self.prob_type_dropdown.value
            params = self._get_params_dict()
            
            # Determine bounds based on probability type
            bound1 = self.bound1_slider.value
            bound2 = self.bound2_slider.value
            
            # Create figure with two subplots (histogram on top)
            fig = make_subplots(
                rows=2, cols=1,
                row_heights=[0.3, 0.7],  # Histogram on top (30%), PDF on bottom (70%)
                vertical_spacing=0.15,
                shared_xaxes=True,
                subplot_titles=('Histogram', 'PDF/PMF and Samples')
            )
            
            # Determine x range
            if len(self.samples) > 0:
                x_min = float(np.min(self.samples)) - 1
                x_max = float(np.max(self.samples)) + 1
            else:
                x_min, x_max = -5, 5
            
            # For discrete, ensure we cover integer values
            if dist_category == "Discrete":
                x_min = max(x_min, 0)
                x_range = np.arange(int(x_min), int(x_max) + 1)
            else:
                x_range = np.linspace(x_min, x_max, 500)
            
            # Only show PDF/PMF if show_pdf is True (plot on bottom, row=2)
            if show_pdf:
                # Compute PDF/PMF
                pdf_pmf_values = compute_pdf_pmf(x_range, dist_type, dist_category, **params)
                
                # Plot PDF/PMF on bottom (row=2)
                if dist_category == "Discrete":
                    # For discrete, plot as bars
                    fig.add_trace(go.Bar(
                        x=x_range,
                        y=pdf_pmf_values,
                        name='PMF',
                        marker=dict(color='lightblue', line=dict(color='blue', width=1)),
                        showlegend=True
                    ), row=2, col=1)
                else:
                    # For continuous, plot as line
                    fig.add_trace(go.Scatter(
                        x=x_range,
                        y=pdf_pmf_values,
                        mode='lines',
                        name='PDF',
                        line=dict(color='blue', width=2),
                        showlegend=True
                    ), row=2, col=1)
                
                # Determine which region to shade based on prob_type
                if prob_type == "of outcome":
                    shade_x = [bound1, bound1]
                    shade_y = [0, np.max(pdf_pmf_values) * 1.1]
                elif prob_type == "under upper bound":
                    mask = x_range <= bound2  # Use upper bound (bound2)
                    shade_x = x_range[mask]
                    shade_y = pdf_pmf_values[mask]
                elif prob_type == "above lower bound":
                    mask = x_range > bound1
                    shade_x = x_range[mask]
                    shade_y = pdf_pmf_values[mask]
                elif prob_type == "in interval":
                    mask = (x_range > bound1) & (x_range <= bound2)
                    shade_x = x_range[mask]
                    shade_y = pdf_pmf_values[mask]
                
                # Add shaded area for PDF/PMF (on bottom, row=2)
                if prob_type == "of outcome":
                    # Vertical line for discrete outcome
                    fig.add_trace(go.Scatter(
                        x=[bound1, bound1],
                        y=[0, np.max(pdf_pmf_values) * 1.1],
                        mode='lines',
                        name='Bound',
                        line=dict(color='red', width=2, dash='dash'),
                        showlegend=False
                    ), row=2, col=1)
                    if dist_category == "Discrete":
                        # Highlight the bar
                        idx = np.argmin(np.abs(x_range - bound1))
                        if idx < len(pdf_pmf_values):
                            fig.add_trace(go.Bar(
                                x=[x_range[idx]],
                                y=[pdf_pmf_values[idx]],
                                name='Selected',
                                marker=dict(color='red', line=dict(color='darkred', width=2)),
                                showlegend=False
                            ), row=2, col=1)
                else:
                    # Shaded area
                    if len(shade_x) > 0:
                        if dist_category == "Discrete":
                            # For discrete, highlight bars
                            mask_idx = np.isin(x_range, shade_x)
                            fig.add_trace(go.Bar(
                                x=x_range[mask_idx],
                                y=pdf_pmf_values[mask_idx],
                                name='Selected Region',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False
                            ), row=2, col=1)
                        else:
                            # For continuous, add filled area
                            fig.add_trace(go.Scatter(
                                x=np.concatenate([[shade_x[0]], shade_x, [shade_x[-1]]]),
                                y=np.concatenate([[0], shade_y, [0]]),
                                fill='tozeroy',
                                mode='lines',
                                name='Selected Region',
                                line=dict(color='rgba(255,0,0,0.3)'),
                                fillcolor='rgba(255,0,0,0.3)',
                                showlegend=False
                            ), row=2, col=1)
                
                # Add vertical lines for bounds (on bottom, row=2)
                if prob_type == "under upper bound":
                    # Show upper bound line
                    fig.add_trace(go.Scatter(
                        x=[bound2, bound2],
                        y=[0, np.max(pdf_pmf_values) * 1.1],
                        mode='lines',
                        name='Upper bound',
                        line=dict(color='red', width=2, dash='dash'),
                        showlegend=False
                    ), row=2, col=1)
                elif prob_type == "above lower bound":
                    # Show lower bound line
                    fig.add_trace(go.Scatter(
                        x=[bound1, bound1],
                        y=[0, np.max(pdf_pmf_values) * 1.1],
                        mode='lines',
                        name='Lower bound',
                        line=dict(color='red', width=2, dash='dash'),
                        showlegend=False
                    ), row=2, col=1)
                elif prob_type == "in interval":
                    # Show both bounds
                    fig.add_trace(go.Scatter(
                        x=[bound1, bound1],
                        y=[0, np.max(pdf_pmf_values) * 1.1],
                        mode='lines',
                        name='Lower bound',
                        line=dict(color='red', width=2, dash='dash'),
                        showlegend=False
                    ), row=2, col=1)
                    fig.add_trace(go.Scatter(
                        x=[bound2, bound2],
                        y=[0, np.max(pdf_pmf_values) * 1.1],
                        mode='lines',
                        name='Upper bound',
                        line=dict(color='red', width=2, dash='dash'),
                        showlegend=False
                    ), row=2, col=1)
            
            # Create histogram (on top, row=1)
            if len(self.samples) > 0:
                if dist_category == "Discrete":
                    # For discrete, use integer bins
                    unique_vals, counts = np.unique(self.samples, return_counts=True)
                    counts = counts / len(self.samples)  # Normalize to probability
                    fig.add_trace(go.Bar(
                        x=unique_vals,
                        y=counts,
                        name='Histogram',
                        marker=dict(color='steelblue', line=dict(color='navy', width=1)),
                        showlegend=True
                    ), row=1, col=1)
                    
                    # Shade histogram for probability region (on top, row=1)
                    if prob_type == "of outcome":
                        mask = unique_vals == int(np.round(bound1))
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=unique_vals[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='red', line=dict(color='darkred', width=2)),
                                showlegend=False
                            ), row=1, col=1)
                    elif prob_type == "under upper bound":
                        mask = unique_vals <= bound2  # Use upper bound (bound2)
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=unique_vals[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False
                            ), row=1, col=1)
                    elif prob_type == "above lower bound":
                        mask = unique_vals > bound1
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=unique_vals[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False
                            ), row=1, col=1)
                    elif prob_type == "in interval":
                        mask = (unique_vals > bound1) & (unique_vals <= bound2)
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=unique_vals[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False
                            ), row=1, col=1)
                else:
                    # For continuous, use regular histogram (on top, row=1)
                    n_bins = 50
                    counts, bin_edges = np.histogram(self.samples, bins=n_bins, range=(x_min, x_max), density=True)
                    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                    fig.add_trace(go.Bar(
                        x=bin_centers,
                        y=counts,
                        name='Histogram',
                        marker=dict(color='steelblue', line=dict(color='navy', width=1)),
                        showlegend=True,
                        width=(bin_edges[1] - bin_edges[0]) * 0.9
                    ), row=1, col=1)
                    
                    # Shade histogram for probability region (on top, row=1)
                    if prob_type == "under upper bound":
                        mask = bin_centers <= bound2  # Use upper bound (bound2)
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=bin_centers[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False,
                                width=(bin_edges[1] - bin_edges[0]) * 0.9
                            ), row=1, col=1)
                    elif prob_type == "above lower bound":
                        mask = bin_centers > bound1
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=bin_centers[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False,
                                width=(bin_edges[1] - bin_edges[0]) * 0.9
                            ), row=1, col=1)
                    elif prob_type == "in interval":
                        mask = (bin_centers > bound1) & (bin_centers <= bound2)
                        if np.any(mask):
                            fig.add_trace(go.Bar(
                                x=bin_centers[mask],
                                y=counts[mask],
                                name='Selected',
                                marker=dict(color='rgba(255,0,0,0.5)', line=dict(color='red', width=2)),
                                showlegend=False,
                                width=(bin_edges[1] - bin_edges[0]) * 0.9
                            ), row=1, col=1)
            
            # Update layout (histogram on top row=1, PDF on bottom row=2)
            fig.update_xaxes(title_text="x", row=1, col=1)
            fig.update_yaxes(title_text="Density", row=1, col=1)
            fig.update_yaxes(title_text="Density", row=2, col=1)
            fig.update_layout(height=700, showlegend=True)
            
            # Compute probabilities
            est_prob = compute_estimated_probability(self.samples, prob_type, bound1, bound2)
            if show_pdf:
                true_prob = compute_true_probability(dist_type, dist_category, prob_type, bound1, bound2, **params)
                # Update probability label
                self.prob_label.value = (
                    f"<b>Estimated Probability (from samples):</b> {est_prob:.4f}<br>"
                    f"<b>True Probability (from {('CDF' if dist_category == 'Continuous' else 'PMF')}):</b> {true_prob:.4f}"
                )
            else:
                # Only show estimated probability when PDF is not shown
                self.prob_label.value = (
                    f"<b>Estimated Probability (from samples):</b> {est_prob:.4f}<br>"
                    f"<b>True Probability:</b> N/A (click 'Draw Samples' to see comparison)"
                )
            
            fig.show()
            
    def display(self):
        """Display the complete interface"""
        # Update parameter widgets initially
        self._update_param_widgets()
        
        # Show blank plot initially
        self._show_blank_plot()
        
        # Create main layout
        controls = widgets.VBox([
            self.category_dropdown,
            self.dist_dropdown,
            self.param_container,
            self.n_samples_slider,
            widgets.HBox([self.draw_button, self.reset_button]),  # Buttons side by side
            self.prob_controls_container  # Probability controls (initially hidden, includes show_pdf_button)
        ])
        
        display(widgets.HBox([controls, self.plot_output]))


In [7]:
# Create and display the interactive visualization
viz = DistributionProbabilityVisualization()
viz.display()
